In [1]:
from keras.models import load_model

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
cd /gdrive/My Drive/lab_project02/

/gdrive/My Drive/lab_project02


In [4]:
ls -l

total 1132509
drwx------ 2 root root      4096 Sep 17 01:33 18.09.16(InceptionResnet_model_tb)/
drwx------ 2 root root      4096 Sep 17 01:41 18.09.17(DenseNet121_model_tb)/
drwx------ 2 root root      4096 Sep 17 09:38 18.09.17(next_fit_model_tb)/
-rw------- 1 root root    151417 Sep 27 04:20 Adam_SGD.ipynb
-rw------- 1 root root  85464296 Sep 17 06:04 DenseNet121_decay_model.h5
-rw------- 1 root root  29120624 Sep 17 07:47 DenseNet121_decay_weights(18_09_17).h5
-rw------- 1 root root  85462112 Sep 17 03:08 DenseNet121_model.h5
-rw------- 1 root root    154206 Sep 30 06:27 DenseNet.ipynb
-rw------- 1 root root    151999 Sep 27 11:27 DenseNet_next_fitting.ipynb
-rw------- 1 root root     19842 Oct  9 17:59 Ensembles.ipynb
-rw------- 1 root root 654333608 Sep 16 14:13 InceptionResNetV2_valchack_model.h5
-rw------- 1 root root 219057808 Sep 16 14:21 InceptionResNetV2_weights(18_09_15).h5
drwx------ 2 root root      4096 Aug 30 01:15 last_img_data/
-rw------- 1 root root     42188 Sep 16 

In [0]:
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img

In [0]:
# dimensions of our images.
img_width, img_height = 579, 238
# img_width, img_height = 150, 150 # VGG 적용

In [0]:
train_data_dir = './split_data/train'
validation_data_dir = './split_data/val'
test_data_dir = './split_data/test'
# validation_data_dir = cwd + '/chest_xray/val'
# test_data_dir = cwd + '/chest_xray/test'

# nb_train_samples = 5217
# nb_validation_samples = 17
epochs = 20
batch_size = 16

In [0]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [0]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
validation_datagen = ImageDataGenerator(rescale=1. / 255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 6397 images belonging to 8 classes.


In [13]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 800 images belonging to 8 classes.


In [14]:
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 800 images belonging to 8 classes.


In [16]:
a = load_model('DenseNet121_decay_model.h5')
b = load_model('InceptionResNetV2_valchack_model.h5')
c = load_model('next_fit_model.h5')

In [0]:
a = load_model('DenseNet121_decay_model.h5')

In [0]:
models = [a, b, c]

In [0]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Average, Dropout
from keras.models import Model, Input

In [0]:
def ensemble(models, model_input):
  
  outputs = [model(model_input) for model in models]
  y = Average()(outputs)
  
  model = Model(model_input, y, name='ensemble')
  
  return model

In [0]:
test_X, test_Y = test_generator.next()

In [0]:
model_input = Input(shape=(img_width, img_height, 3))

In [75]:
print(a.name, b.name, c.name)

model_2 model_1 model_2


In [0]:
a.name = "model_1"

In [0]:
b.name = "model_2"

In [0]:
c.name = "model_3"

In [0]:
ensemble_model = ensemble(models, model_input)

In [0]:
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [85]:
# evaluate the model
import keras.backend.tensorflow_backend as K
with K.tf.device('/gpu:0'):
  scores = ensemble_model.evaluate_generator(test_generator)
print("\n%s: %.2f%%" % (ensemble_model.metrics_names[1], scores[1]*100))


acc: 59.25%


In [86]:
print(scores)

[1.2202400994300842, 0.5925]


In [87]:
ensemble_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 579, 238, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 8)            7045704     input_2[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 8)            54349032    input_2[0][0]                    
__________________________________________________________________________________________________
model_3 (Model)                 (None, 8)            7045704     input_2[0][0]                    
__________________________________________________________________________________________________
average_11